In [20]:
import random
import csv
import time, array

import pandas as pd
from math import factorial



from deap import base, creator, tools, algorithms, benchmarks 

from scipy.spatial.distance import cdist


In [7]:
def mutInversion(individual):
    size = len(individual)
    if size == 0:
        return individual,

    index_one = random.randrange(size)
    index_two = random.randrange(size)
    start_index = min(index_one, index_two)
    end_index = max(index_one, index_two)

    # Reverse the contents of the individual between the indices
    individual[start_index:end_index] = individual[start_index:end_index][::-1]

    return individual,

In [16]:
def measure_time(x):
    execution_time = time.time() - start_time 
#     writer.writerow([execution_time])
    return execution_time

In [22]:
def nsga_ii(toolbox, stats=None, verbose=False):
    pop = toolbox.population(n=toolbox.pop_size)
    pop = toolbox.select(pop, len(pop))
    return algorithms.eaMuPlusLambda(pop, 
                                     toolbox, 
                                     mu=toolbox.pop_size, 
                                     lambda_=toolbox.pop_size, 
                                     cxpb=toolbox.cross_prob,
                                     mutpb=toolbox.mut_prob,
                                     ngen=toolbox.max_gen,
                                     stats=stats, 
                                     verbose=verbose)

In [ ]:
def nsga_iii(toolbox,stats,NGEN,MU,CXPB,MUTPB):
    pop = toolbox.population(n=MU)
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
#   fitnesses = toolbox.map(toolbox.evaluate, pop)        
#     for ind, fit in zip(pop, fitnesses):
#         ind.fitness.values = fit
    
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    pop = toolbox.select(pop, len(pop))
    # Begin the generational process
    for gen in range(1, NGEN):
        offspring = algorithms.varAnd(pop, toolbox, CXPB, MUTPB)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        record = stats.compile(pop)
        print(record)

        # Select the next generation population
        pop = toolbox.select(pop + offspring, MU)
    
    return pop

In [8]:
kroA100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroA100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")
kroB100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroB100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")
kroC100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroC100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")

In [9]:
one = cdist(kroA100.values,kroA100.values)
two = cdist(kroB100.values,kroB100.values)
three = cdist(kroC100.values,kroC100.values)

In [10]:
def motsp(x, one=one, two=two, three=three):
        n_cities = len(x)
        dist_one = 0
        dist_two = 0
        dist_three = 0
        
        for k in range(n_cities - 1):
            i, j = x[k], x[k + 1]
            dist_one += one[i, j]
            dist_two += two[i, j]
            dist_three += three[i, j]

        last, first = x[-1], x[0]
        
        dist_one += one[last, first]
        dist_one = 100000/dist_one
        dist_two += two[last, first]
        dist_two = 100000/dist_two
        dist_three += three[last, first]
        dist_three = 100000/dist_three
        
        return dist_one, dist_two, dist_three

In [23]:
NOBJ = 3
NGEN = 5000
CXPB = 0.8
MUTPB = 0.2

P = 12
H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))
ref_points = tools.uniform_reference_points(NOBJ, P)

MU = 92

creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * NOBJ) 
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=100

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(100), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", motsp) #Fitness Function

toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", mutInversion)
toolbox.register("select", tools.selNSGA2)

toolbox.pop_size = MU
toolbox.max_gen = NGEN
toolbox.mut_prob = MUTPB
toolbox.cross_prob = CXPB

stats = tools.Statistics()
stats.register("time", measure_time)

with open('deap_time_nsga2_motsp_92_500_3_100.csv', 'w', newline='') as file:   
    writer = csv.writer(file)
    writer.writerow(["Execution Time"])
    
    start_time = time.time()
    res, logbook = nsga_ii(toolbox, stats=stats, verbose=True)
    
    fronts = tools.emo.sortLogNondominated(res, len(res))
    
    for i, inds in enumerate(fronts):
        par = [toolbox.evaluate(ind) for ind in inds]
        df = pd.DataFrame(par)
        
    df.to_csv('deap_fitness_nsga2_motsp_92_500_3_100.csv', index=False)

gen	nevals	time     
0  	92    	0.0498657
1  	92    	0.116076 
2  	92    	0.174871 
3  	92    	0.233902 
4  	92    	0.307708 
5  	92    	0.374523 
6  	92    	0.434365 
7  	92    	0.495203 
8  	92    	0.570997 
9  	92    	0.642194 
10 	92    	0.701039 
11 	92    	0.768855 
12 	92    	0.82176  
13 	92    	0.881555 
14 	92    	0.959346 
15 	92    	1.04811  
16 	92    	1.12641  
17 	92    	1.19327  
18 	92    	1.25765  
19 	92    	1.3125   
20 	92    	1.36835  
21 	92    	1.42325  
22 	92    	1.48604  
23 	92    	1.55685  
24 	92    	1.62466  
25 	92    	1.69946  
26 	92    	1.76584  
27 	92    	1.83994  
28 	92    	1.90772  
29 	92    	1.97705  
30 	92    	2.12169  
31 	92    	2.18851  
32 	92    	2.2673   
33 	92    	2.35307  
34 	92    	2.43884  
35 	92    	2.52162  
36 	92    	2.61238  
37 	92    	2.69815  
38 	92    	2.75898  
39 	92    	2.81982  
40 	92    	2.87771  
41 	92    	2.95347  
42 	92    	3.04673  
43 	92    	3.12453  
44 	92    	3.19633  
45 	92    	3.26216  
46 	92    	3.

391	92    	26.1899  
392	92    	26.2598  
393	92    	26.3495  
394	92    	26.4323  
395	92    	26.5071  
396	92    	26.5904  
397	92    	26.6592  
398	92    	26.7175  
399	92    	26.7853  
400	92    	26.8532  
401	92    	26.9275  
402	92    	27.0023  
403	92    	27.0622  
404	92    	27.1449  
405	92    	27.2198  
406	92    	27.2859  
407	92    	27.3657  
408	92    	27.4385  
409	92    	27.4944  
410	92    	27.5532  
411	92    	27.623   
412	92    	27.6939  
413	92    	27.7696  
414	92    	27.8454  
415	92    	27.9212  
416	92    	27.995   
417	92    	28.0708  
418	92    	28.1327  
419	92    	28.1866  
420	92    	28.2394  
421	92    	28.2985  
422	92    	28.3713  
423	92    	28.4446  
424	92    	28.5074  
425	92    	28.5693  
426	92    	28.6272  
427	92    	28.6959  
428	92    	28.7598  
429	92    	28.8186  
430	92    	28.8914  
431	92    	28.9592  
432	92    	29.0335  
433	92    	29.0934  
434	92    	29.1682  
435	92    	29.238   
436	92    	29.3148  
437	92    	29.3937  
438	92    	29

783	92    	51.8588  
784	92    	51.9645  
785	92    	52.0682  
786	92    	52.142   
787	92    	52.2228  
788	92    	52.2787  
789	92    	52.3326  
790	92    	52.4013  
791	92    	52.4781  
792	92    	52.5649  
793	92    	52.6407  
794	92    	52.7155  
795	92    	52.7834  
796	92    	52.8416  
797	92    	52.9115  
798	92    	52.9828  
799	92    	53.0522  
800	92    	53.11    
801	92    	53.1651  
802	92    	53.2194  
803	92    	53.2823  
804	92    	53.3411  
805	92    	53.401   
806	92    	53.4638  
807	92    	53.5326  
808	92    	53.5984  
809	92    	53.6872  
810	92    	53.756   
811	92    	53.8238  
812	92    	53.8926  
813	92    	53.9508  
814	92    	54.0136  
815	92    	54.0794  
816	92    	54.1393  
817	92    	54.2011  
818	92    	54.2595  
819	92    	54.3343  
820	92    	54.4011  
821	92    	54.475   
822	92    	54.5469  
823	92    	54.6187  
824	92    	54.6885  
825	92    	54.7603  
826	92    	54.8181  
827	92    	54.8818  
828	92    	54.9401  
829	92    	54.996   
830	92    	55

1172	92    	76.672   
1173	92    	76.7398  
1174	92    	76.7971  
1175	92    	76.858   
1176	92    	76.9188  
1177	92    	76.9786  
1178	92    	77.0378  
1179	92    	77.0897  
1180	92    	77.1496  
1181	92    	77.2084  
1182	92    	77.2672  
1183	92    	77.332   
1184	92    	77.3869  
1185	92    	77.4567  
1186	92    	77.5335  
1187	92    	77.6003  
1188	92    	77.6781  
1189	92    	77.741   
1190	92    	77.8084  
1191	92    	77.8612  
1192	92    	77.9292  
1193	92    	77.9876  
1194	92    	78.0474  
1195	92    	78.1102  
1196	92    	78.1818  
1197	92    	78.2356  
1198	92    	78.2875  
1199	92    	78.3433  
1200	92    	78.4012  
1201	92    	78.466   
1202	92    	78.5243  
1203	92    	78.5822  
1204	92    	78.635   
1205	92    	78.6979  
1206	92    	78.7587  
1207	92    	78.8145  
1208	92    	78.8664  
1209	92    	78.9282  
1210	92    	78.985   
1211	92    	79.0399  
1212	92    	79.0918  
1213	92    	79.1526  
1214	92    	79.2105  
1215	92    	79.2663  
1216	92    	79.3162  
1217	92   

1548	92    	99.6704  
1549	92    	99.7482  
1550	92    	99.8051  
1551	92    	99.868   
1552	92    	99.9209  
1553	92    	99.9827  
1554	92    	100.047  
1555	92    	100.112  
1556	92    	100.17   
1557	92    	100.235  
1558	92    	100.298  
1559	92    	100.357  
1560	92    	100.408  
1561	92    	100.466  
1562	92    	100.528  
1563	92    	100.583  
1564	92    	100.642  
1565	92    	100.716  
1566	92    	100.784  
1567	92    	100.837  
1568	92    	100.908  
1569	92    	100.962  
1570	92    	101.021  
1571	92    	101.079  
1572	92    	101.142  
1573	92    	101.208  
1574	92    	101.264  
1575	92    	101.325  
1576	92    	101.378  
1577	92    	101.433  
1578	92    	101.49   
1579	92    	101.553  
1580	92    	101.616  
1581	92    	101.682  
1582	92    	101.747  
1583	92    	101.8    
1584	92    	101.851  
1585	92    	101.907  
1586	92    	101.968  
1587	92    	102.028  
1588	92    	102.082  
1589	92    	102.137  
1590	92    	102.191  
1591	92    	102.255  
1592	92    	102.324  
1593	92   

...

3422	92    	219.717  
3423	92    	219.77   
3424	92    	219.846  
3425	92    	219.902  
3426	92    	219.955  
3427	92    	220.017  
3428	92    	220.071  
3429	92    	220.133  
3430	92    	220.202  
3431	92    	220.259  
3432	92    	220.318  
3433	92    	220.393  
3434	92    	220.436  
3435	92    	220.488  
3436	92    	220.557  
3437	92    	220.603  
3438	92    	220.657  
3439	92    	220.719  
3440	92    	220.773  
3441	92    	220.82   
3442	92    	220.889  
3443	92    	220.954  
3444	92    	221.019  
3445	92    	221.073  
3446	92    	221.12   
3447	92    	221.189  
3448	92    	221.236  
3449	92    	221.305  
3450	92    	221.358  
3451	92    	221.405  
3452	92    	221.459  
3453	92    	221.521  
3454	92    	221.575  
3455	92    	221.637  
3456	92    	221.691  
3457	92    	221.737  
3458	92    	221.806  
3459	92    	221.86   
3460	92    	221.922  
3461	92    	221.981  
3462	92    	222.02   
3463	92    	222.091  
3464	92    	222.138  
3465	92    	222.192  
3466	92    	222.254  
3467	92   

3797	92    	241.92   
3798	92    	241.993  
3799	92    	242.059  
3800	92    	242.104  
3801	92    	242.174  
3802	92    	242.225  
3803	92    	242.286  
3804	92    	242.336  
3805	92    	242.444  
3806	92    	242.54   
3807	92    	242.589  
3808	92    	242.636  
3809	92    	242.704  
3810	92    	242.767  
3811	92    	242.82   
3812	92    	242.867  
3813	92    	242.921  
3814	92    	242.973  
3815	92    	243.037  
3816	92    	243.099  
3817	92    	243.156  
3818	92    	243.206  
3819	92    	243.253  
3820	92    	243.322  
3821	92    	243.369  
3822	92    	243.44   
3823	92    	243.497  
3824	92    	243.542  
3825	92    	243.622  
3826	92    	243.685  
3827	92    	243.791  
3828	92    	243.874  
3829	92    	243.929  
3830	92    	243.984  
3831	92    	244.054  
3832	92    	244.108  
3833	92    	244.154  
3834	92    	244.208  
3835	92    	244.255  
3836	92    	244.308  
3837	92    	244.392  
3838	92    	244.473  
3839	92    	244.544  
3840	92    	244.628  
3841	92    	244.709  
3842	92   

4172	92    	264.979  
4173	92    	265.032  
4174	92    	265.104  
4175	92    	265.164  
4176	92    	265.211  
4177	92    	265.282  
4178	92    	265.338  
4179	92    	265.4    
4180	92    	265.464  
4181	92    	265.533  
4182	92    	265.596  
4183	92    	265.649  
4184	92    	265.711  
4185	92    	265.765  
4186	92    	265.834  
4187	92    	265.915  
4188	92    	265.987  
4189	92    	266.054  
4190	92    	266.102  
4191	92    	266.166  
4192	92    	266.235  
4193	92    	266.289  
4194	92    	266.366  
4195	92    	266.421  
4196	92    	266.486  
4197	92    	266.55   
4198	92    	266.602  
4199	92    	266.666  
4200	92    	266.735  
4201	92    	266.797  
4202	92    	266.85   
4203	92    	266.913  
4204	92    	266.966  
4205	92    	267.036  
4206	92    	267.084  
4207	92    	267.152  
4208	92    	267.199  
4209	92    	267.252  
4210	92    	267.319  
4211	92    	267.368  
4212	92    	267.439  
4213	92    	267.484  
4214	92    	267.552  
4215	92    	267.599  
4216	92    	267.668  
4217	92   

4546	92    	287.103  
4547	92    	287.151  
4548	92    	287.214  
4549	92    	287.277  
4550	92    	287.33   
4551	92    	287.399  
4552	92    	287.462  
4553	92    	287.515  
4554	92    	287.589  
4555	92    	287.657  
4556	92    	287.735  
4557	92    	287.792  
4558	92    	287.848  
4559	92    	287.901  
4560	92    	287.948  
4561	92    	288.017  
4562	92    	288.086  
4563	92    	288.156  
4564	92    	288.222  
4565	92    	288.281  
4566	92    	288.333  
4567	92    	288.386  
4568	92    	288.451  
4569	92    	288.502  
4570	92    	288.565  
4571	92    	288.634  
4572	92    	288.703  
4573	92    	288.77   
4574	92    	288.818  
4575	92    	288.887  
4576	92    	288.934  
4577	92    	289.003  
4578	92    	289.066  
4579	92    	289.119  
4580	92    	289.188  
4581	92    	289.251  
4582	92    	289.304  
4583	92    	289.368  
4584	92    	289.417  
4585	92    	289.489  
4586	92    	289.55   
4587	92    	289.589  
4588	92    	289.652  
4589	92    	289.719  
4590	92    	289.784  
4591	92   

4919	92    	310.854  
4920	92    	310.918  
4921	92    	310.983  
4922	92    	311.049  
4923	92    	311.112  
4924	92    	311.165  
4925	92    	311.218  
4926	92    	311.297  
4927	92    	311.35   
4928	92    	311.397  
4929	92    	311.451  
4930	92    	311.519  
4931	92    	311.566  
4932	92    	311.62   
4933	92    	311.682  
4934	92    	311.735  
4935	92    	311.798  
4936	92    	311.867  
4937	92    	311.92   
4938	92    	311.985  
4939	92    	312.036  
4940	92    	312.083  
4941	92    	312.152  
4942	92    	312.198  
4943	92    	312.251  
4944	92    	312.316  
4945	92    	312.368  
4946	92    	312.421  
4947	92    	312.484  
4948	92    	312.537  
4949	92    	312.584  
4950	92    	312.653  
4951	92    	312.7    
4952	92    	312.788  
4953	92    	312.859  
4954	92    	312.934  
4955	92    	313.003  
4956	92    	313.076  
4957	92    	313.139  
4958	92    	313.201  
4959	92    	313.254  
4960	92    	313.323  
4961	92    	313.417  
4962	92    	313.471  
4963	92    	313.524  
4964	92   

In [ ]:
NOBJ = 3
NGEN = 5000
CXPB = 0.8
MUTPB = 0.2

P = 12
H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))

MU = int(H + (4 - H % 4))

ref_points = tools.uniform_reference_points(NOBJ, P)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * NOBJ) 
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=100

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(100), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", motsp) #Fitness Function

toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", mutInversion)
toolbox.register("select", tools.selNSGA3, ref_points=ref_points)

stats = tools.Statistics()
stats.register("time", measure_time)


with open('deap_time_nsga3_motsp_92_500_3_100.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Execution Time"])
    
    
    start_time = time.time()
    res = nsga_iii(toolbox,stats,NGEN,MU,CXPB,MUTPB)
    
    fronts = tools.emo.sortLogNondominated(res, len(res))
    
    for i, inds in enumerate(fronts):
        par = [toolbox.evaluate(ind) for ind in inds]
        df = pd.DataFrame(par)
        
    df.to_csv('deap_fitness_nsga3_motsp_92_500_3_100.csv', index=False)